In [2]:
from __future__ import absolute_import, division, print_function

import os 
import cv2
import numpy as np

import torch 
from torch.utils.data import DataLoader

from layers import disp_to_depth
from utils import readlines
from options import ModelOptions
import datasets
import networks
import matplotlib.pyplot as plt
from scipy import stats as scpystats

%matplotlib inline 


In [4]:
model_name = 'all_tasks_dwa_cs'
weights_path = 'weights/' + model_name
tasks = ['pose', 'depth', 'instance']

In [9]:
output_root = 'results/'
output_dir = output_root + model_name
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    

In [16]:
data_path = 'data/'
images = os.listdir(data_path)


In [18]:
encoder_path = os.path.join(weights_path, 'encoder.pth')
decoder_path = os.path.join(weights_path, 'decoder.pth')

depth_att_enc_path = []
depth_att_dec_path = []
depth_att_enc = []
depth_att_dec = []

ins_att_enc_path = []
ins_att_dec_path = []
ins_att_enc = []
ins_att_dec = []

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

attention_channels_enc = np.array([64, 128, 256, 512])
attention_channels_dec = np.array([16, 32, 64, 128, 256,512])


for i in range(5):
    if i < 4:
        depth_att_enc_path.append(os.path.join(weights_path, '(\'depth_att_enc\', ' + str(i) + ').pth'))
        ins_att_enc_path.append(os.path.join(weights_path, '(\'instance_att_enc\', ' + str(i) + ').pth'))
        if i == 0:
            depth_att_enc.append(networks.AttentionEncoder(attention_channels_enc[i],
                                                            attention_channels_enc[i], do_pool=True))
            ins_att_enc.append(networks.AttentionEncoder(attention_channels_enc[i],
                                                            attention_channels_enc[i], do_pool=True))
        else:
            depth_att_enc.append(
                networks.AttentionEncoder(attention_channels_enc[i] + attention_channels_enc[i - 1],
                                            attention_channels_enc[i], do_pool=False))
            ins_att_enc.append(networks.AttentionEncoder(attention_channels_enc[i] + attention_channels_enc[i - 1],
                                                            attention_channels_enc[i], do_pool=False))
        ins_att_enc[i].load_state_dict(torch.load(ins_att_enc_path[i]))
        depth_att_enc[i].load_state_dict(torch.load(depth_att_enc_path[i]))

    depth_att_dec_path.append(os.path.join(weights_path, '(\'depth_att_dec\', ' + str(i) + ').pth'))
    ins_att_dec_path.append(os.path.join(weights_path, '(\'instance_att_dec\', ' + str(i) + ').pth'))
    if i < 2:
        depth_att_dec.append(
            networks.AttentionDecoder(attention_channels_dec[-i - 1], attention_channels_dec[-i - 2],
                                        do_upsample=False))
        ins_att_dec.append(networks.AttentionDecoder(attention_channels_dec[-i - 1], attention_channels_dec[-i - 2],
                                                        do_upsample=False))
    else:
        depth_att_dec.append(
            networks.AttentionDecoder(attention_channels_dec[-i - 1], attention_channels_dec[-i - 2],
                                        do_upsample=True))
        ins_att_dec.append(networks.AttentionDecoder(attention_channels_dec[-i - 1], attention_channels_dec[-i - 2],
                                                        do_upsample=True))
    depth_att_dec[i].load_state_dict(torch.load(depth_att_dec_path[i]))
    ins_att_dec[i].load_state_dict(torch.load(ins_att_dec_path[i]))

    att_disp_3_path = os.path.join(weights_path, '(\'att_disp\', 0).pth')
    att_disp_3 = networks.AttentionDispDecoder(0)
    att_disp_3.load_state_dict(torch.load(att_disp_3_path))

att_ins_0_path = os.path.join(weights_path, '(\'att_ins\', 0).pth')
att_ins_0 = networks.AttentionInsDecoder(0, num_output_channels=2)
att_ins_0.load_state_dict(torch.load(att_ins_0_path))

encoder_dict = torch.load(encoder_path, map_location=device)

encoder = networks.ResnetEncoderMtan(18, False)
decoder = networks.SharedDecoder(encoder.num_ch_enc, num_output_channels=2)

model_dict = encoder.state_dict()
encoder.load_state_dict({k: v for k,v in encoder_dict.items() if k in model_dict})
decoder.load_state_dict(torch.load(decoder_path))

#encoder.cuda()
encoder.eval()
#decoder.cuda()
decoder.eval()



RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location='cpu' to map your storages to the CPU.